In [1]:
import simpy
import random

# With recording waiting times

In [18]:
class M_M_n_queue:
    def __init__(self, env, arrival_rate, service_rate, num_servers):
        self.env = env
        self.server = simpy.Resource(env, num_servers)
        self.arrival_rate = arrival_rate
        self.service_rate = service_rate
        self.waiting_time_in_the_queue=[]
        

    def arrival(self):
        """The arrival process generates customers according to a Poisson process"""
        customer = 0
        while True:
            ### Interarrival times follow exponential
            next_customer= random.expovariate(self.arrival_rate)
            ### Wait for the customer
            yield self.env.timeout(next_customer)  
            customer += 1
            
            ### Start a service process
            self.env.process(self.service(customer))

    def service(self, customer):
        """The service process. The customer arrives and requests a service. The service time follows 
           the exponential distribution"""
        with self.server.request() as req:
            queue_arrival_time= self.env.now
            yield req
            #### calculate waiting time
            waiting_time = self.env.now - queue_arrival_time
            
            service_time = random.expovariate(self.service_rate)
            yield self.env.timeout(service_time)
            
            self.waiting_time_in_the_queue.append(waiting_time)
        
def setup(arrival_rate, service_rate, sim_time, num_servers):
    """Setup and start the simulation and returns average waiting time in one simulation"""
    env = simpy.Environment()
    M_M_n = M_M_n_queue(env, arrival_rate, service_rate, num_servers)
    env.process(M_M_n.arrival())
    env.run(until=sim_time)
    average_waiting_time = sum(M_M_n.waiting_time_in_the_queue) / len(M_M_n.waiting_time_in_the_queue)

    return average_waiting_time, len(M_M_n.waiting_time_in_the_queue)

def simulate(arrival_rate, service_rate, sim_time, num_simulations, num_servers):
    """Returns average waiting time over provided number of simulations"""
    average_waiting_time_list = []
    for i in range(num_simulations):
        average_waiting_time, N_customers = setup(arrival_rate, service_rate, sim_time, num_servers)
        average_waiting_time_list.append(average_waiting_time)
        
    average_waiting_time_over_simulations = sum(average_waiting_time_list) / num_simulations

    print(f' Average waiting time in the queue with number of servers ={num_servers} over {num_simulations} simulations: {average_waiting_time_over_simulations}')
    return average_waiting_time_over_simulations, N_customers



In [19]:
# Parameters 

arrival_rate = 5  #average time between arrivals is 0.2 minutes, or on average 5 people arrive every minute, so the rate=5
service_rate = 1/5   # average service time is 5 minutes, so the rate is 1/5 

simulation_time = 1000  
num_simulations = 1000
num_servers= 1

simulate(arrival_rate, service_rate, simulation_time, num_simulations, num_servers)


 Average waiting time in the queue with number of servers =1 over 1000 simulations: 475.36534244912286


(475.36534244912286, 198)

# For different number of servers n

In [21]:
num_servers_list= [1, 2, 4]

for num_servers in num_servers_list:
    _, N_customers = simulate(arrival_rate, service_rate, simulation_time, num_simulations, num_servers)
    print(f'Customers: {N_customers} \n')


 Average waiting time in the queue with number of servers =1 over 1000 simulations: 475.2852871388753
Customers: 193 

 Average waiting time in the queue with number of servers =2 over 1000 simulations: 454.3343269161822
Customers: 401 

 Average waiting time in the queue with number of servers =4 over 1000 simulations: 415.454546423222
Customers: 789 

